# Bragg peak over time – self-contained notebook

All calculations are in this notebook (no imports from the analysis script). Loads only the HDF keys needed for Bragg peak and intensity vs time.

**How to run:** From `emilio_scripts` or `emilio_scripts/notebooks`. In the paths cell: set `RUN` (e.g. 73 → A073) and choose `BASE_DIR` (Desktop or volume).

## 1. Paths and config

In [2]:
from pathlib import Path
import sys
import matplotlib
matplotlib.use("macosx")  # interactive plots in separate windows

# emilio_scripts only: find python_scripts from cwd (emilio_scripts or emilio_scripts/notebooks)
if (Path.cwd() / "python_scripts").exists():
    EMILIO_SCRIPTS = Path.cwd()
else:
    EMILIO_SCRIPTS = Path.cwd().parent
SCRIPTS = EMILIO_SCRIPTS / "python_scripts"
sys.path.insert(0, str(SCRIPTS))

# Optional: add source if any notebook code needs it
SOURCE = EMILIO_SCRIPTS / "source"
if SOURCE.exists():
    sys.path.insert(0, str(SOURCE))

# --- Run and base directory ---
RUN = 73  # e.g. 73 -> A073
RUN_ID = f"A{RUN:03d}"

BASE_DIR = Path("/Users/emilioescauriza/Desktop")
# BASE_DIR = Path("/Volumes/EmilioSD4TB/APS_08-IDEI-2025-1006")

DATA_DIR = BASE_DIR / "Twotime_PostExpt_01"
RESULTS_HDF = DATA_DIR / f"{RUN_ID}_IPA_NBH_1_att0100_260K_001_results.hdf"
NPZ_PATH = DATA_DIR / f"{RUN_ID}_inferred_qphi_maps.npz"

print("SCRIPTS:", SCRIPTS)
print("RUN_ID:", RUN_ID)
print("RESULTS_HDF:", RESULTS_HDF, "exists:", RESULTS_HDF.exists())
print("NPZ_PATH:", NPZ_PATH, "exists:", NPZ_PATH.exists())

SCRIPTS: /Users/emilioescauriza/Documents/repos/006_APS_8IDE/emilio_scripts/python_scripts
RUN_ID: A073
RESULTS_HDF: /Users/emilioescauriza/Desktop/Twotime_PostExpt_01/A073_IPA_NBH_1_att0100_260K_001_results.hdf exists: True
NPZ_PATH: /Users/emilioescauriza/Desktop/Twotime_PostExpt_01/A073_inferred_qphi_maps.npz exists: True


## 2. Load image data

In [3]:
import h5py
import numpy as np

DATA_LOADED = RESULTS_HDF.exists() and NPZ_PATH.exists()
if not DATA_LOADED:
    scattering_2d = intensity_vs_time = q_map = phi_map = None
    valid_mask = None
    print("Data not loaded: RESULTS_HDF or NPZ_PATH not found.")
    print("  RESULTS_HDF:", RESULTS_HDF, "exists:", RESULTS_HDF.exists())
    print("  NPZ_PATH:", NPZ_PATH, "exists:", NPZ_PATH.exists())
    print("Edit the first code cell, set both paths to your files, then re-run from cell 1.")
else:
    with h5py.File(RESULTS_HDF, "r") as f:
        scattering_2d = np.asarray(f["xpcs/temporal_mean/scattering_2d"][...])
        intensity_vs_time = np.asarray(f["xpcs/spatial_mean/intensity_vs_time"][...])
    if scattering_2d.ndim == 3:
        scattering_2d = scattering_2d[0]
    d = np.load(NPZ_PATH, allow_pickle=False)
    q_map = d["q_map"]
    phi_map = d["phi_map"]
    valid_mask = d.get("valid_mask", None)
    print("scattering_2d.shape:", scattering_2d.shape, "| intensity_vs_time.shape:", intensity_vs_time.shape, "| q_map.shape:", q_map.shape)

scattering_2d.shape: (1676, 2100) | intensity_vs_time.shape: (2, 4800) | q_map.shape: (1676, 2100)


## 3. List HDF keys to find more time-resolved data

In [4]:
def list_keys(name, obj):
    print(name)

if not DATA_LOADED:
    print("Skipped (no data). Set RESULTS_HDF and NPZ_PATH in the first cell and re-run from cell 1.")
else:
    with h5py.File(RESULTS_HDF, "r") as f:
        f.visititems(list_keys)

entry
entry/beamline
entry/definition
entry/end_time
entry/entry_identifier
entry/entry_identifier_uuid
entry/instrument
entry/instrument/attenuator_1
entry/instrument/attenuator_1/attenuator_index
entry/instrument/attenuator_1/attenuator_transmission
entry/instrument/attenuator_2
entry/instrument/attenuator_2/attenuator_index
entry/instrument/attenuator_2/attenuator_transmission
entry/instrument/beam_stop
entry/instrument/beam_stop/size
entry/instrument/beam_stop/x_position
entry/instrument/beam_stop/y_position
entry/instrument/bluesky
entry/instrument/bluesky/bluesky_plan
entry/instrument/bluesky/bluesky_plan_kwargs
entry/instrument/bluesky/bluesky_version
entry/instrument/bluesky/parent_folder
entry/instrument/bluesky/scan_id
entry/instrument/bluesky/spec_file
entry/instrument/datamanagement
entry/instrument/datamanagement/workflow_kwargs
entry/instrument/datamanagement/workflow_name
entry/instrument/datamanagement/workflow_version
entry/instrument/detector_1
entry/instrument/detect

## 4. Skew normal peak fit

In [11]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy import stats as scipy_stats
from scipy import optimize as scipy_optimize

def _fit_skewnorm_weighted_1d(x, w, half_win=None, eps=1e-12):
    x = np.asarray(x, dtype=np.float64).ravel()
    w = np.clip(np.asarray(w, dtype=np.float64).ravel(), 0.0, None)
    sw = float(np.sum(w))
    if x.size == 0 or sw <= 0:
        return {"loc": np.nan, "scale": np.nan, "shape": 0.0}
    mu = float(np.sum(w * x) / sw)
    if half_win is not None:
        mask = np.abs(x - mu) <= half_win
        x_win, w_win = x[mask].copy(), w[mask].copy()
        sw_win = float(np.sum(w_win))
        if sw_win > eps and x_win.size > 0:
            x, w, sw = x_win, w_win, sw_win
            mu = float(np.sum(w * x) / sw)
    sig = float(np.sqrt(max(float(np.sum(w * (x - mu) ** 2) / sw), 0.0)))
    if sig <= 0:
        return {"loc": mu, "scale": 0.0, "shape": 0.0}
    skew_emp = float(np.clip(np.sum(w * (x - mu) ** 3) / sw / (sig ** 3 + eps), -0.99, 0.99))
    def obj(a):
        return float(scipy_stats.skewnorm.stats(a, moments="s")) - skew_emp
    # Narrow bracket: moderate skew only (a in [-4, 4] ~ skewness in ~[-0.7, 0.7])
    a_lo, a_hi = -4.0, 4.0
    if obj(a_lo) * obj(a_hi) < 0:
        a_fit = float(scipy_optimize.brentq(obj, a_lo, a_hi, xtol=1e-6))
    else:
        a_fit = a_hi if skew_emp > 0.03 else (a_lo if skew_emp < -0.03 else 0.0)
    m1 = float(scipy_stats.skewnorm.stats(a_fit, moments="m"))
    m2_sn = float(scipy_stats.skewnorm.stats(a_fit, moments="v"))
    scale = sig / (np.sqrt(m2_sn) + eps)
    loc = mu - scale * m1
    return {"loc": loc, "scale": scale, "shape": a_fit}

if not DATA_LOADED:
    print("Skipped (no data). Run the load cell above first.")
else:
    # Crop 500x500 box around brightest pixel
    half = 150
    iy_max, ix_max = np.unravel_index(np.nanargmax(scattering_2d), scattering_2d.shape)
    H, W = scattering_2d.shape
    y_lo = max(0, iy_max - half)
    y_hi = min(H, iy_max + half)
    x_lo = max(0, ix_max - half)
    x_hi = min(W, ix_max + half)
    img_crop = scattering_2d[y_lo:y_hi, x_lo:x_hi]
    valid = np.isfinite(img_crop) & (img_crop > 0)
    # Tighter percentiles = more contrast (less washed out). Tune p_lo, p_hi if needed.
    p_lo, p_hi = 10.0, 99.99
    vmin = np.nanpercentile(img_crop[valid], p_lo)
    vmax = np.nanpercentile(img_crop[valid], p_hi)
    if not np.isfinite(vmin):
        vmin = np.nanmin(img_crop[valid]) if np.any(valid) else 1e-10
    if not np.isfinite(vmax):
        vmax = np.nanmax(img_crop[valid]) if np.any(valid) else vmin + 1.0
    if vmax <= vmin or vmin <= 0:
        vmin = max(1e-10, np.nanmin(img_crop[valid])) if np.any(valid) else 1e-10
        vmax = max(vmin + 1.0, np.nanmax(img_crop[valid])) if np.any(valid) else vmin + 1.0
    # Lineout length: smaller = fit focused on peak core (narrower window)
    half_ln = 40
    x_lo_ln = max(0, ix_max - half_ln)
    x_hi_ln = min(W, ix_max + half_ln)
    y_lo_ln = max(0, iy_max - half_ln)
    y_hi_ln = min(H, iy_max + half_ln)
    lineout_x = np.asarray(scattering_2d[iy_max, x_lo_ln:x_hi_ln], dtype=float)
    lineout_y = np.asarray(scattering_2d[y_lo_ln:y_hi_ln, ix_max], dtype=float)
    wx = np.clip(lineout_x, 0.0, None)
    wy = np.clip(lineout_y, 0.0, None)
    fit_half_win = 18  # fit moments only within ±this of peak centre → narrower width
    fit_x = _fit_skewnorm_weighted_1d(np.arange(len(lineout_x)), wx, half_win=fit_half_win)
    fit_y = _fit_skewnorm_weighted_1d(np.arange(len(lineout_y)), wy, half_win=fit_half_win)
    from matplotlib.gridspec import GridSpec
    fig = plt.figure(figsize=(16, 8))
    gs = GridSpec(2, 2, height_ratios=[1.2, 1], hspace=0.35, wspace=0.3)
    ax_img = fig.add_subplot(gs[0, :])
    ax_x = fig.add_subplot(gs[1, 0])
    ax_y = fig.add_subplot(gs[1, 1])
    im = ax_img.imshow(img_crop, origin="upper", cmap="magma", norm=LogNorm(vmin=vmin, vmax=vmax), interpolation="nearest", aspect="equal")
    ax_img.set_title("Temporal mean scattering (log scale)")
    ax_img.set_xticks([])
    ax_img.set_yticks([])
    cy_crop = iy_max - y_lo
    cx_crop = ix_max - x_lo
    ax_img.axhline(y=cy_crop, color="cyan", lw=0.8, alpha=0.9)
    ax_img.axvline(x=cx_crop, color="cyan", lw=0.8, alpha=0.9)
    fig.colorbar(im, ax=ax_img, label="Intensity")
    xx = np.arange(len(lineout_x))
    ax_x.plot(xx, lineout_x, "C0-", lw=1, label="data")
    if np.isfinite(fit_x["loc"]) and fit_x["scale"] > 0:
        pdf_x = scipy_stats.skewnorm.pdf(xx, fit_x["shape"], fit_x["loc"], fit_x["scale"])
        scale_x = np.nanmax(lineout_x) / (np.nanmax(pdf_x) + 1e-12)
        ax_x.plot(xx, pdf_x * scale_x, "C1-", lw=1.2, label="skew-normal fit")
    ax_x.set_ylabel("Intensity")
    ax_x.set_title(f"X lineout ({str(half_ln)} px)")
    ax_x.legend(loc="upper right", fontsize=8)
    ax_x.grid(True, alpha=0.3)
    yy = np.arange(len(lineout_y))
    ax_y.plot(yy, lineout_y, "C0-", lw=1, label="data")
    if np.isfinite(fit_y["loc"]) and fit_y["scale"] > 0:
        pdf_y = scipy_stats.skewnorm.pdf(yy, fit_y["shape"], fit_y["loc"], fit_y["scale"])
        scale_y = np.nanmax(lineout_y) / (np.nanmax(pdf_y) + 1e-12)
        ax_y.plot(yy, pdf_y * scale_y, "C1-", lw=1.2, label="skew-normal fit")
    ax_y.set_xlabel("Pixel index")
    ax_y.set_ylabel("Intensity")
    ax_y.set_title(f"Y lineout ({str(half_ln)} px)")
    ax_y.legend(loc="upper right", fontsize=8)
    ax_y.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

/var/folders/rf/wmzlz2kd779618zr2mgpkhhc0000gn/T/ipykernel_88968/3357500712.py:111: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


KeyboardInterrupt: 

## 5. Half-widths at half max (HWHM_L, HWHM_R)

Lineouts through the brightest pixel with horizontal line at half maximum. Left and right half-widths at half maximum (HWHM_L, HWHM_R) capture asymmetry; FWHM = HWHM_L + HWHM_R.

In [110]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import numpy as np

def _fwhm_1d(x, y):
    """Return (left_cross, right_cross, fwhm) in x units; uses linear interpolation at half max."""
    y = np.asarray(y, dtype=np.float64)
    peak_idx = int(np.nanargmax(y))
    half_max = y[peak_idx] / 2.0
    if half_max <= 0 or not np.isfinite(half_max):
        return np.nan, np.nan, np.nan
    x = np.asarray(x, dtype=np.float64)
    n = len(y)
    # Left crossing: first i < peak_idx where y[i] <= half_max, then interp
    left_cross = np.nan
    for i in range(peak_idx - 1, -1, -1):
        if y[i] <= half_max:
            if i + 1 < n and y[i + 1] > y[i]:
                t = (half_max - y[i]) / (y[i + 1] - y[i] + 1e-12)
                left_cross = x[i] + t * (x[i + 1] - x[i])
            else:
                left_cross = float(x[i])
            break
        if i == 0:
            left_cross = float(x[0])
    # Right crossing
    right_cross = np.nan
    for i in range(peak_idx + 1, n):
        if y[i] <= half_max:
            if i - 1 >= 0 and y[i - 1] > y[i]:
                t = (half_max - y[i]) / (y[i - 1] - y[i] + 1e-12)
                right_cross = x[i] + t * (x[i - 1] - x[i])
            else:
                right_cross = float(x[i])
            break
        if i == n - 1:
            right_cross = float(x[n - 1])
    fwhm = (right_cross - left_cross) if (np.isfinite(left_cross) and np.isfinite(right_cross)) else np.nan
    return left_cross, right_cross, fwhm

if not DATA_LOADED:
    print("Skipped (no data). Run the load cell above first.")
else:
    half = 250
    iy_max, ix_max = np.unravel_index(np.nanargmax(scattering_2d), scattering_2d.shape)
    H, W = scattering_2d.shape
    y_lo = max(0, iy_max - half)
    y_hi = min(H, iy_max + half)
    x_lo = max(0, ix_max - half)
    x_hi = min(W, ix_max + half)
    img_crop = scattering_2d[y_lo:y_hi, x_lo:x_hi]
    valid = np.isfinite(img_crop) & (img_crop > 0)
    p_lo, p_hi = 10.0, 99.99
    vmin = np.nanpercentile(img_crop[valid], p_lo)
    vmax = np.nanpercentile(img_crop[valid], p_hi)
    if not np.isfinite(vmin): vmin = np.nanmin(img_crop[valid]) if np.any(valid) else 1e-10
    if not np.isfinite(vmax): vmax = np.nanmax(img_crop[valid]) if np.any(valid) else vmin + 1.0
    if vmax <= vmin or vmin <= 0:
        vmin = max(1e-10, np.nanmin(img_crop[valid])) if np.any(valid) else 1e-10
        vmax = max(vmin + 1.0, np.nanmax(img_crop[valid])) if np.any(valid) else vmin + 1.0
    half_ln = 40
    x_lo_ln = max(0, ix_max - half_ln)
    x_hi_ln = min(W, ix_max + half_ln)
    y_lo_ln = max(0, iy_max - half_ln)
    y_hi_ln = min(H, iy_max + half_ln)
    lineout_x = np.asarray(scattering_2d[iy_max, x_lo_ln:x_hi_ln], dtype=float)
    lineout_y = np.asarray(scattering_2d[y_lo_ln:y_hi_ln, ix_max], dtype=float)
    xx = np.arange(len(lineout_x))
    yy = np.arange(len(lineout_y))
    lx, rx, fwhm_x = _fwhm_1d(xx, lineout_x)
    ly, ry, fwhm_y = _fwhm_1d(yy, lineout_y)
    peak_x = float(xx[np.nanargmax(lineout_x)])
    peak_y = float(yy[np.nanargmax(lineout_y)])
    hwhm_l_x = peak_x - lx if np.isfinite(lx) else np.nan
    hwhm_r_x = rx - peak_x if np.isfinite(rx) else np.nan
    hwhm_l_y = peak_y - ly if np.isfinite(ly) else np.nan
    hwhm_r_y = ry - peak_y if np.isfinite(ry) else np.nan
    half_max_x = np.nanmax(lineout_x) / 2.0
    half_max_y = np.nanmax(lineout_y) / 2.0
    fig = plt.figure(figsize=(16, 8))
    gs = GridSpec(2, 2, height_ratios=[1.2, 1], hspace=0.35, wspace=0.3)
    ax_img = fig.add_subplot(gs[0, :])
    ax_x = fig.add_subplot(gs[1, 0])
    ax_y = fig.add_subplot(gs[1, 1])
    im = ax_img.imshow(img_crop, origin="upper", cmap="magma", norm=LogNorm(vmin=vmin, vmax=vmax), interpolation="nearest", aspect="equal")
    ax_img.set_title("Temporal mean scattering (log scale)")
    ax_img.set_xticks([])
    ax_img.set_yticks([])
    cy_crop = iy_max - y_lo
    cx_crop = ix_max - x_lo
    ax_img.axhline(y=cy_crop, color="cyan", lw=0.8, alpha=0.9)
    ax_img.axvline(x=cx_crop, color="cyan", lw=0.8, alpha=0.9)
    fig.colorbar(im, ax=ax_img, label="Intensity")
    ax_x.plot(xx, lineout_x, "C0-", lw=1, label="data")
    ax_x.axhline(half_max_x, color="gray", ls="--", lw=0.8, alpha=0.8)
    if np.isfinite(fwhm_x):
        ax_x.axvline(lx, color="gray", ls=":", lw=0.6)
        ax_x.axvline(rx, color="gray", ls=":", lw=0.6)
        ax_x.annotate(f"HWHM_L = {hwhm_l_x:.1f} px  HWHM_R = {hwhm_r_x:.1f} px", xy=((lx + rx) / 2, half_max_x), fontsize=9, ha="center", va="bottom")
        ax_x.annotate(f"FWHM = {fwhm_x:.1f} px", xy=((lx + rx) / 2, half_max_x * 0.85), fontsize=9, ha="center", va="bottom")
    ax_x.set_ylabel("Intensity")
    ax_x.set_title("X lineout")
    ax_x.legend(loc="upper right", fontsize=8)
    ax_x.grid(True, alpha=0.3)
    ax_y.plot(yy, lineout_y, "C1-", lw=1, label="data")
    ax_y.axhline(half_max_y, color="gray", ls="--", lw=0.8, alpha=0.8)
    if np.isfinite(fwhm_y):
        ax_y.axvline(ly, color="gray", ls=":", lw=0.6)
        ax_y.axvline(ry, color="gray", ls=":", lw=0.6)
        ax_y.annotate(f"HWHM_L = {hwhm_l_y:.1f} px  HWHM_R = {hwhm_r_y:.1f} px", xy=((ly + ry) / 2, half_max_y), fontsize=9, ha="center", va="bottom")
        ax_y.annotate(f"FWHM = {fwhm_y:.1f} px", xy=((ly + ry) / 2, half_max_y * 0.85), fontsize=9, ha="center", va="bottom")
    ax_y.set_xlabel("Pixel index")
    ax_y.set_ylabel("Intensity")
    ax_y.set_title("Y lineout")
    ax_y.legend(loc="upper right", fontsize=8)
    ax_y.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

/var/folders/rf/wmzlz2kd779618zr2mgpkhhc0000gn/T/ipykernel_74650/1826906984.py:118: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## 6. Split Gaussian

Fit a split (two-sided) Gaussian to each lineout: left of centre uses σ_L, right of centre uses σ_R. Continuous at the peak; captures asymmetry via different widths.

In [8]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import numpy as np
from scipy.optimize import curve_fit

def _split_gaussian(x, A, mu, sigma_L, sigma_R):
    """Split Gaussian: left of mu uses sigma_L, right uses sigma_R. Continuous at peak."""
    x = np.asarray(x, dtype=np.float64)
    left = x <= mu
    out = np.empty_like(x)
    out[left] = A * np.exp(-0.5 * ((x[left] - mu) / np.clip(sigma_L, 1e-6, None)) ** 2)
    out[~left] = A * np.exp(-0.5 * ((x[~left] - mu) / np.clip(sigma_R, 1e-6, None)) ** 2)
    return out

def _fit_split_gaussian_1d(x, y):
    """Fit split Gaussian; return (A, mu, sigma_L, sigma_R) or None on failure."""
    y = np.asarray(y, dtype=np.float64)
    x = np.asarray(x, dtype=np.float64)
    peak_idx = int(np.nanargmax(y))
    A0 = float(np.nanmax(y))
    mu0 = float(x[peak_idx])
    sig0 = max(2.0, (np.nanmax(x) - np.nanmin(x)) / 8.0)
    try:
        popt, _ = curve_fit(
            _split_gaussian, x, np.clip(y, 0, None),
            p0=[A0, mu0, sig0, sig0],
            bounds=([0, x.min(), 0.5, 0.5], [np.nanmax(y) * 2, x.max(), 1e4, 1e4]),
            maxfev=5000,
        )
        return popt
    except Exception:
        return None

if not DATA_LOADED:
    print("Skipped (no data). Run the load cell above first.")
else:
    half = 250
    iy_max, ix_max = np.unravel_index(np.nanargmax(scattering_2d), scattering_2d.shape)
    H, W = scattering_2d.shape
    y_lo = max(0, iy_max - half)
    y_hi = min(H, iy_max + half)
    x_lo = max(0, ix_max - half)
    x_hi = min(W, ix_max + half)
    img_crop = scattering_2d[y_lo:y_hi, x_lo:x_hi]
    valid = np.isfinite(img_crop) & (img_crop > 0)
    p_lo, p_hi = 10.0, 99.99
    vmin = np.nanpercentile(img_crop[valid], p_lo)
    vmax = np.nanpercentile(img_crop[valid], p_hi)
    if not np.isfinite(vmin): vmin = np.nanmin(img_crop[valid]) if np.any(valid) else 1e-10
    if not np.isfinite(vmax): vmax = np.nanmax(img_crop[valid]) if np.any(valid) else vmin + 1.0
    if vmax <= vmin or vmin <= 0:
        vmin = max(1e-10, np.nanmin(img_crop[valid])) if np.any(valid) else 1e-10
        vmax = max(vmin + 1.0, np.nanmax(img_crop[valid])) if np.any(valid) else vmin + 1.0
    half_ln = 40
    x_lo_ln = max(0, ix_max - half_ln)
    x_hi_ln = min(W, ix_max + half_ln)
    y_lo_ln = max(0, iy_max - half_ln)
    y_hi_ln = min(H, iy_max + half_ln)
    lineout_x = np.asarray(scattering_2d[iy_max, x_lo_ln:x_hi_ln], dtype=float)
    lineout_y = np.asarray(scattering_2d[y_lo_ln:y_hi_ln, ix_max], dtype=float)
    xx = np.arange(len(lineout_x))
    yy = np.arange(len(lineout_y))
    fit_x = _fit_split_gaussian_1d(xx, lineout_x)
    fit_y = _fit_split_gaussian_1d(yy, lineout_y)
    fig = plt.figure(figsize=(16, 8))
    gs = GridSpec(2, 2, height_ratios=[1.2, 1], hspace=0.35, wspace=0.3)
    ax_img = fig.add_subplot(gs[0, :])
    ax_x = fig.add_subplot(gs[1, 0])
    ax_y = fig.add_subplot(gs[1, 1])
    im = ax_img.imshow(img_crop, origin="upper", cmap="magma", norm=LogNorm(vmin=vmin, vmax=vmax), interpolation="nearest", aspect="equal")
    ax_img.set_title("Temporal mean scattering (log scale)")
    ax_img.set_xticks([])
    ax_img.set_yticks([])
    cy_crop = iy_max - y_lo
    cx_crop = ix_max - x_lo
    ax_img.axhline(y=cy_crop, color="cyan", lw=0.8, alpha=0.9)
    ax_img.axvline(x=cx_crop, color="cyan", lw=0.8, alpha=0.9)
    fig.colorbar(im, ax=ax_img, label="Intensity")
    ax_x.plot(xx, lineout_x, "C0-", lw=1, label="data")
    if fit_x is not None:
        ax_x.plot(xx, _split_gaussian(xx, *fit_x), "C1-", lw=1.2, label="split Gaussian")
        ax_x.annotate(f"σ_L = {fit_x[2]:.2f}  σ_R = {fit_x[3]:.2f}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=9, va="top")
    ax_x.set_ylabel("Intensity")
    ax_x.set_title("X lineout")
    ax_x.legend(loc="upper right", fontsize=8)
    ax_x.grid(True, alpha=0.3)
    ax_y.plot(yy, lineout_y, "C0-", lw=1, label="data")
    if fit_y is not None:
        ax_y.plot(yy, _split_gaussian(yy, *fit_y), "C1-", lw=1.2, label="split Gaussian")
        ax_y.annotate(f"σ_L = {fit_y[2]:.2f}  σ_R = {fit_y[3]:.2f}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=9, va="top")
    ax_y.set_xlabel("Pixel index")
    ax_y.set_ylabel("Intensity")
    ax_y.set_title("Y lineout")
    ax_y.legend(loc="upper right", fontsize=8)
    ax_y.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

/var/folders/rf/wmzlz2kd779618zr2mgpkhhc0000gn/T/ipykernel_88968/762418135.py:97: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## 7. Moment-based descriptors

Weighted mean (centre), standard deviation (width), skewness, and optionally kurtosis from the lineout. Optional overlay of a skew-normal with the same moments.

In [112]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import numpy as np
from scipy import stats as scipy_stats

def _weighted_moments_1d(x, w, half_win=None, eps=1e-12):
    """Return dict with mean, std, skewness, kurtosis (weighted). Optionally restrict to window around peak."""
    x = np.asarray(x, dtype=np.float64).ravel()
    w = np.clip(np.asarray(w, dtype=np.float64).ravel(), 0.0, None)
    sw = float(np.sum(w))
    if x.size == 0 or sw <= 0:
        return {"mean": np.nan, "std": np.nan, "skew": np.nan, "kurt": np.nan}
    mu = float(np.sum(w * x) / sw)
    if half_win is not None:
        mask = np.abs(x - mu) <= half_win
        x, w = x[mask], w[mask]
        sw = float(np.sum(w))
        if sw <= eps:
            return {"mean": mu, "std": np.nan, "skew": np.nan, "kurt": np.nan}
        mu = float(np.sum(w * x) / sw)
    var = float(np.sum(w * (x - mu) ** 2) / sw)
    std = np.sqrt(max(var, 0.0))
    if std <= eps:
        return {"mean": mu, "std": 0.0, "skew": 0.0, "kurt": 0.0}
    m3 = float(np.sum(w * (x - mu) ** 3) / sw)
    m4 = float(np.sum(w * (x - mu) ** 4) / sw)
    skew = m3 / (std ** 3 + eps)
    kurt = m4 / (std ** 4 + eps) - 3.0  # excess kurtosis
    return {"mean": mu, "std": std, "skew": skew, "kurt": kurt}

if not DATA_LOADED:
    print("Skipped (no data). Run the load cell above first.")
else:
    half = 250
    iy_max, ix_max = np.unravel_index(np.nanargmax(scattering_2d), scattering_2d.shape)
    H, W = scattering_2d.shape
    y_lo = max(0, iy_max - half)
    y_hi = min(H, iy_max + half)
    x_lo = max(0, ix_max - half)
    x_hi = min(W, ix_max + half)
    img_crop = scattering_2d[y_lo:y_hi, x_lo:x_hi]
    valid = np.isfinite(img_crop) & (img_crop > 0)
    p_lo, p_hi = 10.0, 99.99
    vmin = np.nanpercentile(img_crop[valid], p_lo)
    vmax = np.nanpercentile(img_crop[valid], p_hi)
    if not np.isfinite(vmin): vmin = np.nanmin(img_crop[valid]) if np.any(valid) else 1e-10
    if not np.isfinite(vmax): vmax = np.nanmax(img_crop[valid]) if np.any(valid) else vmin + 1.0
    if vmax <= vmin or vmin <= 0:
        vmin = max(1e-10, np.nanmin(img_crop[valid])) if np.any(valid) else 1e-10
        vmax = max(vmin + 1.0, np.nanmax(img_crop[valid])) if np.any(valid) else vmin + 1.0
    half_ln = 40
    fit_half_win = 18
    x_lo_ln = max(0, ix_max - half_ln)
    x_hi_ln = min(W, ix_max + half_ln)
    y_lo_ln = max(0, iy_max - half_ln)
    y_hi_ln = min(H, iy_max + half_ln)
    lineout_x = np.asarray(scattering_2d[iy_max, x_lo_ln:x_hi_ln], dtype=float)
    lineout_y = np.asarray(scattering_2d[y_lo_ln:y_hi_ln, ix_max], dtype=float)
    wx = np.clip(lineout_x, 0.0, None)
    wy = np.clip(lineout_y, 0.0, None)
    xx = np.arange(len(lineout_x))
    yy = np.arange(len(lineout_y))
    mom_x = _weighted_moments_1d(xx, wx, half_win=fit_half_win)
    mom_y = _weighted_moments_1d(yy, wy, half_win=fit_half_win)
    # Map skewness to skew-normal shape parameter (approx) for overlay
    def _skew_to_shape(skew, eps=1e-12):
        from scipy import optimize
        def obj(a):
            return float(scipy_stats.skewnorm.stats(a, moments="s")) - np.clip(skew, -0.99, 0.99)
        try:
            if obj(-4) * obj(4) < 0:
                return float(optimize.brentq(obj, -4, 4, xtol=1e-6))
        except Exception:
            pass
        return 0.0
    ax_fit_x = _skew_to_shape(mom_x["skew"])
    ax_fit_y = _skew_to_shape(mom_y["skew"])
    m1_x = float(scipy_stats.skewnorm.stats(ax_fit_x, moments="m"))
    m2_x = float(scipy_stats.skewnorm.stats(ax_fit_x, moments="v"))
    scale_x = mom_x["std"] / (np.sqrt(m2_x) + 1e-12)
    loc_x = mom_x["mean"] - scale_x * m1_x
    m1_y = float(scipy_stats.skewnorm.stats(ax_fit_y, moments="m"))
    m2_y = float(scipy_stats.skewnorm.stats(ax_fit_y, moments="v"))
    scale_y = mom_y["std"] / (np.sqrt(m2_y) + 1e-12)
    loc_y = mom_y["mean"] - scale_y * m1_y
    fig = plt.figure(figsize=(16, 8))
    gs = GridSpec(2, 2, height_ratios=[1.2, 1], hspace=0.35, wspace=0.3)
    ax_img = fig.add_subplot(gs[0, :])
    ax_x = fig.add_subplot(gs[1, 0])
    ax_y = fig.add_subplot(gs[1, 1])
    im = ax_img.imshow(img_crop, origin="upper", cmap="magma", norm=LogNorm(vmin=vmin, vmax=vmax), interpolation="nearest", aspect="equal")
    ax_img.set_title("Temporal mean scattering (log scale)")
    ax_img.set_xticks([])
    ax_img.set_yticks([])
    cy_crop = iy_max - y_lo
    cx_crop = ix_max - x_lo
    ax_img.axhline(y=cy_crop, color="cyan", lw=0.8, alpha=0.9)
    ax_img.axvline(x=cx_crop, color="cyan", lw=0.8, alpha=0.9)
    fig.colorbar(im, ax=ax_img, label="Intensity")
    ax_x.plot(xx, lineout_x, "C0-", lw=1, label="data")
    if mom_x["std"] > 0:
        pdf_x = scipy_stats.skewnorm.pdf(xx, ax_fit_x, loc_x, scale_x)
        scale_pdf_x = np.nanmax(lineout_x) / (np.nanmax(pdf_x) + 1e-12)
        ax_x.plot(xx, pdf_x * scale_pdf_x, "C1-", lw=1.2, label="skew-normal (moments)")
        ax_x.annotate(f"μ={mom_x['mean']:.1f} σ={mom_x['std']:.2f} skew={mom_x['skew']:.3f}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=9, va="top")
    ax_x.set_ylabel("Intensity")
    ax_x.set_title("X lineout")
    ax_x.legend(loc="upper right", fontsize=8)
    ax_x.grid(True, alpha=0.3)
    ax_y.plot(yy, lineout_y, "C0-", lw=1, label="data")
    if mom_y["std"] > 0:
        pdf_y = scipy_stats.skewnorm.pdf(yy, ax_fit_y, loc_y, scale_y)
        scale_pdf_y = np.nanmax(lineout_y) / (np.nanmax(pdf_y) + 1e-12)
        ax_y.plot(yy, pdf_y * scale_pdf_y, "C1-", lw=1.2, label="skew-normal (moments)")
        ax_y.annotate(f"μ={mom_y['mean']:.1f} σ={mom_y['std']:.2f} skew={mom_y['skew']:.3f}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=9, va="top")
    ax_y.set_xlabel("Pixel index")
    ax_y.set_ylabel("Intensity")
    ax_y.set_title("Y lineout")
    ax_y.legend(loc="upper right", fontsize=8)
    ax_y.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

/var/folders/rf/wmzlz2kd779618zr2mgpkhhc0000gn/T/ipykernel_74650/1998290997.py:122: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## 8. Voigt and Gaussian-tail models

**Voigt:** Convolution of Gaussian and Lorentzian (symmetric); common for instrument-broadened peaks. **Gaussian tail:** Gaussian core with an exponential tail on one side to capture asymmetry.

In [107]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import numpy as np
from scipy.optimize import curve_fit
from scipy.special import voigt_profile

def _voigt(x, A, mu, sigma, gamma):
    """Voigt profile: A * voigt_profile(x - mu, sigma, gamma). sigma=Gaussian, gamma=Lorentzian HWHM."""
    return A * voigt_profile(x - mu, sigma, gamma)

def _gaussian_tail(x, A, mu, sigma, tau_right):
    """Gaussian core; for x > mu use Gaussian decay, for x > mu + 2*sigma use exponential tail (right side)."""
    x = np.asarray(x, dtype=np.float64)
    core = A * np.exp(-0.5 * ((x - mu) / np.clip(sigma, 1e-6, None)) ** 2)
    # Exponential tail on the right: from mu + 2*sigma, decay as exp(-(x - x0)/tau)
    x0 = mu + 2.0 * sigma
    tail_amp = A * np.exp(-0.5 * (2.0 * sigma / np.clip(sigma, 1e-6, None)) ** 2)
    right = x > x0
    tau = np.clip(tau_right, 0.5, 1e4)
    out = core.copy()
    out[right] = tail_amp * np.exp(-(x[right] - x0) / tau)
    return out

def _fit_voigt_1d(x, y):
    """Fit Voigt; return (A, mu, sigma, gamma) or None."""
    y = np.asarray(y, dtype=np.float64)
    x = np.asarray(x, dtype=np.float64)
    peak_idx = int(np.nanargmax(y))
    A0 = float(np.nanmax(y))
    mu0 = float(x[peak_idx])
    sig0 = max(1.0, (np.nanmax(x) - np.nanmin(x)) / 10.0)
    try:
        popt, _ = curve_fit(
            _voigt, x, np.clip(y, 0, None),
            p0=[A0, mu0, sig0, sig0 * 0.5],
            bounds=([0, x.min(), 0.2, 0.01], [np.nanmax(y) * 2, x.max(), 1e3, 1e3]),
            maxfev=5000,
        )
        return popt
    except Exception:
        return None

def _fit_gaussian_tail_1d(x, y):
    """Fit Gaussian + right exponential tail; return (A, mu, sigma, tau_right) or None."""
    y = np.asarray(y, dtype=np.float64)
    x = np.asarray(x, dtype=np.float64)
    peak_idx = int(np.nanargmax(y))
    A0 = float(np.nanmax(y))
    mu0 = float(x[peak_idx])
    sig0 = max(1.0, (np.nanmax(x) - np.nanmin(x)) / 10.0)
    try:
        popt, _ = curve_fit(
            _gaussian_tail, x, np.clip(y, 0, None),
            p0=[A0, mu0, sig0, 3.0],
            bounds=([0, x.min(), 0.2, 0.2], [np.nanmax(y) * 2, x.max(), 1e3, 1e4]),
            maxfev=5000,
        )
        return popt
    except Exception:
        return None

if not DATA_LOADED:
    print("Skipped (no data). Run the load cell above first.")
else:
    half = 250
    iy_max, ix_max = np.unravel_index(np.nanargmax(scattering_2d), scattering_2d.shape)
    H, W = scattering_2d.shape
    y_lo = max(0, iy_max - half)
    y_hi = min(H, iy_max + half)
    x_lo = max(0, ix_max - half)
    x_hi = min(W, ix_max + half)
    img_crop = scattering_2d[y_lo:y_hi, x_lo:x_hi]
    valid = np.isfinite(img_crop) & (img_crop > 0)
    p_lo, p_hi = 10.0, 99.99
    vmin = np.nanpercentile(img_crop[valid], p_lo)
    vmax = np.nanpercentile(img_crop[valid], p_hi)
    if not np.isfinite(vmin): vmin = np.nanmin(img_crop[valid]) if np.any(valid) else 1e-10
    if not np.isfinite(vmax): vmax = np.nanmax(img_crop[valid]) if np.any(valid) else vmin + 1.0
    if vmax <= vmin or vmin <= 0:
        vmin = max(1e-10, np.nanmin(img_crop[valid])) if np.any(valid) else 1e-10
        vmax = max(vmin + 1.0, np.nanmax(img_crop[valid])) if np.any(valid) else vmin + 1.0
    half_ln = 40
    x_lo_ln = max(0, ix_max - half_ln)
    x_hi_ln = min(W, ix_max + half_ln)
    y_lo_ln = max(0, iy_max - half_ln)
    y_hi_ln = min(H, iy_max + half_ln)
    lineout_x = np.asarray(scattering_2d[iy_max, x_lo_ln:x_hi_ln], dtype=float)
    lineout_y = np.asarray(scattering_2d[y_lo_ln:y_hi_ln, ix_max], dtype=float)
    xx = np.arange(len(lineout_x))
    yy = np.arange(len(lineout_y))
    voigt_x = _fit_voigt_1d(xx, lineout_x)
    voigt_y = _fit_voigt_1d(yy, lineout_y)
    tail_x = _fit_gaussian_tail_1d(xx, lineout_x)
    tail_y = _fit_gaussian_tail_1d(yy, lineout_y)
    fig = plt.figure(figsize=(16, 8))
    gs = GridSpec(2, 2, height_ratios=[1.2, 1], hspace=0.35, wspace=0.3)
    ax_img = fig.add_subplot(gs[0, :])
    ax_x = fig.add_subplot(gs[1, 0])
    ax_y = fig.add_subplot(gs[1, 1])
    im = ax_img.imshow(img_crop, origin="upper", cmap="magma", norm=LogNorm(vmin=vmin, vmax=vmax), interpolation="nearest", aspect="equal")
    ax_img.set_title("Temporal mean scattering (log scale)")
    ax_img.set_xticks([])
    ax_img.set_yticks([])
    cy_crop = iy_max - y_lo
    cx_crop = ix_max - x_lo
    ax_img.axhline(y=cy_crop, color="cyan", lw=0.8, alpha=0.9)
    ax_img.axvline(x=cx_crop, color="cyan", lw=0.8, alpha=0.9)
    fig.colorbar(im, ax=ax_img, label="Intensity")
    ax_x.plot(xx, lineout_x, "C0-", lw=1, label="data")
    if voigt_x is not None:
        ax_x.plot(xx, _voigt(xx, *voigt_x), "C1-", lw=1.2, label="Voigt")
        ax_x.annotate(f"Voigt σ={voigt_x[2]:.2f} γ={voigt_x[3]:.2f}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=8, va="top")
    if tail_x is not None:
        ax_x.plot(xx, _gaussian_tail(xx, *tail_x), "C2--", lw=1, label="Gauss+tail")
        ax_x.annotate(f"tail τ={tail_x[3]:.1f}", xy=(0.05, 0.88), xycoords="axes fraction", fontsize=8, va="top")
    ax_x.set_ylabel("Intensity")
    ax_x.set_title("X lineout")
    ax_x.legend(loc="upper right", fontsize=8)
    ax_x.grid(True, alpha=0.3)
    ax_y.plot(yy, lineout_y, "C0-", lw=1, label="data")
    if voigt_y is not None:
        ax_y.plot(yy, _voigt(yy, *voigt_y), "C1-", lw=1.2, label="Voigt")
        ax_y.annotate(f"Voigt σ={voigt_y[2]:.2f} γ={voigt_y[3]:.2f}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=8, va="top")
    if tail_y is not None:
        ax_y.plot(yy, _gaussian_tail(yy, *tail_y), "C2--", lw=1, label="Gauss+tail")
        ax_y.annotate(f"tail τ={tail_y[3]:.1f}", xy=(0.05, 0.88), xycoords="axes fraction", fontsize=8, va="top")
    ax_y.set_xlabel("Pixel index")
    ax_y.set_ylabel("Intensity")
    ax_y.set_title("Y lineout")
    ax_y.legend(loc="upper right", fontsize=8)
    ax_y.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

/var/folders/rf/wmzlz2kd779618zr2mgpkhhc0000gn/T/ipykernel_74650/3408899202.py:133: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## 9. Area under each side of the peak

Integrate intensity left and right of the peak centre. The **left area** and **right area** give an asymmetry measure: how much of the total flux lies on each side. Ratio (e.g. area_R / area_L) or normalized difference indicates tilt; symmetric peaks have equal areas.

In [178]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import numpy as np

def _area_left_right_1d(x, y):
    """Return (area_left, area_right, total_area, peak_idx). Peak at peak_idx; left = [0..peak], right = [peak..-1]. Uses trapezoidal rule."""
    y = np.asarray(y, dtype=np.float64)
    x = np.asarray(x, dtype=np.float64)
    peak_idx = int(np.nanargmax(y))
    n = len(y)
    # Left: from first point up to and including peak
    x_left = x[: peak_idx + 1]
    y_left = np.clip(y[: peak_idx + 1], 0, None)
    area_left = float(np.trapezoid(y_left, x_left)) if len(x_left) > 1 else 0.0
    # Right: from peak to last point
    x_right = x[peak_idx:]
    y_right = np.clip(y[peak_idx:], 0, None)
    area_right = float(np.trapezoid(y_right, x_right)) if len(x_right) > 1 else 0.0
    total = area_left + area_right
    return area_left, area_right, total, peak_idx

if not DATA_LOADED:
    print("Skipped (no data). Run the load cell above first.")
else:
    half = 250
    iy_max, ix_max = np.unravel_index(np.nanargmax(scattering_2d), scattering_2d.shape)
    H, W = scattering_2d.shape
    y_lo = max(0, iy_max - half)
    y_hi = min(H, iy_max + half)
    x_lo = max(0, ix_max - half)
    x_hi = min(W, ix_max + half)
    img_crop = scattering_2d[y_lo:y_hi, x_lo:x_hi]
    valid = np.isfinite(img_crop) & (img_crop > 0)
    p_lo, p_hi = 10.0, 99.99
    vmin = np.nanpercentile(img_crop[valid], p_lo)
    vmax = np.nanpercentile(img_crop[valid], p_hi)
    if not np.isfinite(vmin): vmin = np.nanmin(img_crop[valid]) if np.any(valid) else 1e-10
    if not np.isfinite(vmax): vmax = np.nanmax(img_crop[valid]) if np.any(valid) else vmin + 1.0
    if vmax <= vmin or vmin <= 0:
        vmin = max(1e-10, np.nanmin(img_crop[valid])) if np.any(valid) else 1e-10
        vmax = max(vmin + 1.0, np.nanmax(img_crop[valid])) if np.any(valid) else vmin + 1.0
    half_ln = 40
    x_lo_ln = max(0, ix_max - half_ln)
    x_hi_ln = min(W, ix_max + half_ln)
    y_lo_ln = max(0, iy_max - half_ln)
    y_hi_ln = min(H, iy_max + half_ln)
    lineout_x = np.asarray(scattering_2d[iy_max, x_lo_ln:x_hi_ln], dtype=float)
    lineout_y = np.asarray(scattering_2d[y_lo_ln:y_hi_ln, ix_max], dtype=float)
    xx = np.arange(len(lineout_x))
    yy = np.arange(len(lineout_y))
    area_l_x, area_r_x, total_x, peak_ix = _area_left_right_1d(xx, lineout_x)
    area_l_y, area_r_y, total_y, peak_iy = _area_left_right_1d(yy, lineout_y)
    ratio_x = area_r_x / (area_l_x + 1e-30)
    ratio_y = area_r_y / (area_l_y + 1e-30)
    asym_norm_x = (area_r_x - area_l_x) / (total_x + 1e-30)
    asym_norm_y = (area_r_y - area_l_y) / (total_y + 1e-30)
    peak_x = float(xx[peak_ix])
    peak_y = float(yy[peak_iy])
    fig = plt.figure(figsize=(16, 8))
    gs = GridSpec(2, 2, height_ratios=[1.2, 1], hspace=0.35, wspace=0.3)
    ax_img = fig.add_subplot(gs[0, :])
    ax_x = fig.add_subplot(gs[1, 0])
    ax_y = fig.add_subplot(gs[1, 1])
    im = ax_img.imshow(img_crop, origin="upper", cmap="magma", norm=LogNorm(vmin=vmin, vmax=vmax), interpolation="nearest", aspect="equal")
    ax_img.set_title("Temporal mean scattering (log scale)")
    ax_img.set_xticks([])
    ax_img.set_yticks([])
    cy_crop = iy_max - y_lo
    cx_crop = ix_max - x_lo
    ax_img.axhline(y=cy_crop, color="cyan", lw=0.8, alpha=0.9)
    ax_img.axvline(x=cx_crop, color="cyan", lw=0.8, alpha=0.9)
    fig.colorbar(im, ax=ax_img, label="Intensity")
    ax_x.plot(xx, lineout_x, "C0-", lw=1, label="data")
    ax_x.axvline(peak_x, color="gray", ls="--", lw=0.8, alpha=0.8, label="peak")
    ax_x.fill_between(xx[xx <= peak_x], 0, lineout_x[xx <= peak_x], color="C0", alpha=0.25)
    ax_x.fill_between(xx[xx >= peak_x], 0, lineout_x[xx >= peak_x], color="C1", alpha=0.25)
    ax_x.annotate(f"area_L = {area_l_x:.0f}  area_R = {area_r_x:.0f}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=9, va="top")
    ax_x.annotate(f"ratio R/L = {ratio_x:.3f}  asym = {asym_norm_x:.3f}", xy=(0.05, 0.85), xycoords="axes fraction", fontsize=9, va="top")
    ax_x.set_xlabel("Pixel index")
    ax_x.set_ylabel("Intensity")
    ax_x.set_title("X lineout")
    ax_x.legend(loc="upper right", fontsize=8)
    ax_x.grid(True, alpha=0.3)
    ax_y.plot(yy, lineout_y, "C1-", lw=1, label="data")
    ax_y.axvline(peak_y, color="gray", ls="--", lw=0.8, alpha=0.8, label="peak")
    ax_y.fill_between(yy[yy <= peak_y], 0, lineout_y[yy <= peak_y], color="C0", alpha=0.25)
    ax_y.fill_between(yy[yy >= peak_y], 0, lineout_y[yy >= peak_y], color="C1", alpha=0.25)
    ax_y.annotate(f"area_L = {area_l_y:.0f}  area_R = {area_r_y:.0f}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=9, va="top")
    ax_y.annotate(f"ratio R/L = {ratio_y:.3f}  asym = {asym_norm_y:.3f}", xy=(0.05, 0.85), xycoords="axes fraction", fontsize=9, va="top")
    ax_y.set_xlabel("Pixel index")
    ax_y.set_ylabel("Intensity")
    ax_y.set_title("Y lineout")
    ax_y.legend(loc="upper right", fontsize=8)
    ax_y.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

/var/folders/rf/wmzlz2kd779618zr2mgpkhhc0000gn/T/ipykernel_74650/583113625.py:96: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## 10. Infer smooth (x,y) → (q, φ) mapping

Fit a geometry-like model from the existing ROI-based q_map and phi_map: φ = atan2(y − cy, x − cx) + φ0 and q = f(r) with r = √[(x−cx)² + (y−cy)²]. The lineouts cell below uses this smooth mapping so q and φ along the contour are continuous.

In [205]:
import numpy as np
from scipy.optimize import least_squares

# Will be set below when DATA_LOADED; cell 11 (lineouts) uses it.
SMOOTH_QPHI_MODEL = None

if not DATA_LOADED:
    print("Skipped (no data). Run the load cell above first.")
else:
    ny, nx = q_map.shape
    yy, xx = np.mgrid[0:ny, 0:nx]
    valid = np.isfinite(q_map) & np.isfinite(phi_map)
    if valid_mask is not None and valid_mask.ndim == 2:
        valid = valid & valid_mask
    n_valid = int(np.sum(valid))
    if n_valid < 10:
        print("Too few valid pixels to fit smooth mapping.")
    else:
        x_flat = xx[valid].astype(np.float64)
        y_flat = yy[valid].astype(np.float64)
        q_flat = q_map[valid].ravel()
        phi_flat = phi_map[valid].ravel()

        # Initial guess: brightest pixel as beam center
        iy_max, ix_max = np.unravel_index(np.nanargmax(scattering_2d), scattering_2d.shape)
        cx0, cy0 = float(ix_max), float(iy_max)
        phi0_0 = float(np.nanmedian(phi_flat))

        def _phi_residual(params):
            cx, cy, phi0 = params[0], params[1], params[2]
            phi_model = np.arctan2(y_flat - cy, x_flat - cx) + phi0
            # Circular residual in [-pi, pi]
            d = phi_flat - phi_model
            return np.arctan2(np.sin(d), np.cos(d))

        fit = least_squares(_phi_residual, [cx0, cy0, phi0_0], loss="soft_l1", f_scale=0.5)
        cx_fit, cy_fit, phi0_fit = fit.x[0], fit.x[1], fit.x[2]

        r_flat = np.sqrt((x_flat - cx_fit) ** 2 + (y_flat - cy_fit) ** 2)
        # q = f(r): polynomial in r (deg 2 or 3)
        deg = min(3, max(1, n_valid // 100))
        q_coeffs = np.polyfit(r_flat, q_flat, deg=deg)
        def q_of_r(r):
            r = np.asarray(r, dtype=np.float64)
            return np.polyval(q_coeffs, np.maximum(r, 0.0))

        def smooth_q_phi_at(x, y):
            """(x, y) in global image coords -> (q, phi)."""
            x, y = np.asarray(x), np.asarray(y)
            r = np.sqrt((x - cx_fit) ** 2 + (y - cy_fit) ** 2)
            phi = np.arctan2(y - cy_fit, x - cx_fit) + phi0_fit
            return q_of_r(r), phi

        SMOOTH_QPHI_MODEL = {
            "cx": cx_fit,
            "cy": cy_fit,
            "phi0": phi0_fit,
            "q_of_r": q_of_r,
            "q_coeffs": q_coeffs,
            "smooth_q_phi_at": smooth_q_phi_at,
        }
        print(f"Inferred beam center (global px): cx={cx_fit:.2f}, cy={cy_fit:.2f}, phi0={phi0_fit:.4f}, q(r) poly deg={deg}")

Inferred beam center (global px): cx=4373.15, cy=3477.73, phi0=-10.1083, q(r) poly deg=3


## 11. Lineouts in q and φ

The brightest pixel has a unique (q, φ). Two paths through that point: **φ-arc** (constant φ, tangent ⊥ ∇φ) and **q-radius** (constant q, tangent ⊥ ∇q). Image on top shows these single paths (cyan = φ-arc, orange = q-radius); below, I(q) along the φ-arc and I(φ) along the q-radius. q and φ along the path use the smooth mapping from section 10 when available.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import numpy as np

def _to_n2(s):
    if isinstance(s, (tuple, list)) and len(s) == 2:
        return np.column_stack([np.asarray(s[0]).ravel(), np.asarray(s[1]).ravel()])
    s = np.asarray(s)
    if s.ndim == 1:
        s = s.reshape(-1, 2)
    return s

def _contour_segments_at_level(Z, level):
    """Get all contour segments at Z=level. Returns list of (n, 2) arrays (x, y) in crop coords."""
    import contourpy
    ny, nx = Z.shape
    X = np.arange(nx)
    Y = np.arange(ny)
    cgen = contourpy.contour_generator(X, Y, Z)
    segs = list(cgen.lines(level))
    if not segs:
        z_min, z_max = np.nanmin(Z), np.nanmax(Z)
        eps = max((z_max - z_min) * 1e-4, 1e-12)
        for lev in [level - eps, level + eps, level - 2*eps, level + 2*eps]:
            if z_min <= lev <= z_max:
                segs = list(cgen.lines(lev))
                if segs:
                    break
    if not segs:
        return []
    return [_to_n2(s) for s in segs if len(np.asarray(s).ravel()) >= 4]

def _contour_segment_through_peak(Z, level, cx, cy):
    """Get the single contour segment at Z=level that passes through (cx, cy). Returns (x, y) array or None."""
    segs = _contour_segments_at_level(Z, level)
    if not segs:
        return None
    dists = [np.min(np.hypot(s[:, 0] - cx, s[:, 1] - cy)) for s in segs]
    return segs[np.argmin(dists)]

def _sample_along_segment(xy, q_crop, phi_crop, img_crop, interpolate=True):
    """xy is (n, 2) in crop coords. Return (q, phi, I) at each point. If interpolate=True use bilinear for one value per contour point."""
    from scipy.ndimage import map_coordinates
    ny, nx = q_crop.shape
    if interpolate and len(xy) > 0:
        # Bilinear: map_coordinates expects (row, col) = (y, x), order 1 = linear
        coords = np.array([xy[:, 1], xy[:, 0]])  # (2, n) as (y_vals, x_vals)
        q_vals = map_coordinates(q_crop, coords, order=1, mode="nearest")
        phi_vals = map_coordinates(phi_crop, coords, order=1, mode="nearest")
        I_vals = map_coordinates(img_crop, coords, order=1, mode="nearest")
        return q_vals.ravel().astype(float), phi_vals.ravel().astype(float), I_vals.ravel().astype(float)
    q_vals, phi_vals, I_vals = [], [], []
    for i in range(len(xy)):
        x, y = xy[i, 0], xy[i, 1]
        ix = int(np.clip(np.round(x), 0, nx - 1))
        iy = int(np.clip(np.round(y), 0, ny - 1))
        q_vals.append(float(q_crop[iy, ix]))
        phi_vals.append(float(phi_crop[iy, ix]))
        I_vals.append(float(img_crop[iy, ix]))
    return np.array(q_vals), np.array(phi_vals), np.array(I_vals)

def _sample_along_segment_smooth(xy, x_lo, y_lo, img_crop, model):
    """xy is (n, 2) in crop coords. Use smooth (x,y)->(q,phi) from model; I from bilinear on img_crop."""
    from scipy.ndimage import map_coordinates
    if len(xy) == 0:
        return np.array([]), np.array([]), np.array([])
    x_global = xy[:, 0] + x_lo
    y_global = xy[:, 1] + y_lo
    q_vals, phi_vals = model["smooth_q_phi_at"](x_global, y_global)
    coords = np.array([xy[:, 1], xy[:, 0]])
    I_vals = map_coordinates(img_crop, coords, order=1, mode="nearest").ravel().astype(float)
    return np.asarray(q_vals, dtype=np.float64), np.asarray(phi_vals, dtype=np.float64), I_vals

if not DATA_LOADED:
    print("Skipped (no data). Run the load cell above first.")
else:
    half = 250
    iy_max, ix_max = np.unravel_index(np.nanargmax(scattering_2d), scattering_2d.shape)
    H, W = scattering_2d.shape
    y_lo = max(0, iy_max - half)
    y_hi = min(H, iy_max + half)
    x_lo = max(0, ix_max - half)
    x_hi = min(W, ix_max + half)
    img_crop = scattering_2d[y_lo:y_hi, x_lo:x_hi]
    q_crop = q_map[y_lo:y_hi, x_lo:x_hi].astype(np.float64)
    phi_crop = phi_map[y_lo:y_hi, x_lo:x_hi].astype(np.float64)
    valid = np.isfinite(img_crop) & (img_crop > 0) & np.isfinite(q_crop) & np.isfinite(phi_crop)
    if valid_mask is not None and valid_mask.ndim == 2:
        valid = valid & valid_mask[y_lo:y_hi, x_lo:x_hi]
    p_lo, p_hi = 10.0, 99.99
    vmin = np.nanpercentile(img_crop[valid], p_lo)
    vmax = np.nanpercentile(img_crop[valid], p_hi)
    if not np.isfinite(vmin): vmin = np.nanmin(img_crop[valid]) if np.any(valid) else 1e-10
    if not np.isfinite(vmax): vmax = np.nanmax(img_crop[valid]) if np.any(valid) else vmin + 1.0
    if vmax <= vmin or vmin <= 0:
        vmin = max(1e-10, np.nanmin(img_crop[valid])) if np.any(valid) else 1e-10
        vmax = max(vmin + 1.0, np.nanmax(img_crop[valid])) if np.any(valid) else vmin + 1.0
    q_peak = float(q_map[iy_max, ix_max])
    phi_peak = float(phi_map[iy_max, ix_max])
    cx, cy = ix_max - x_lo, iy_max - y_lo
    try:
        _smooth_model = SMOOTH_QPHI_MODEL
    except NameError:
        _smooth_model = None
    use_smooth = _smooth_model is not None
    ny, nx = phi_crop.shape
    r_max = np.hypot(nx, ny) * 2.0
    if use_smooth:
        cx_center = _smooth_model["cx"] - x_lo
        cy_center = _smooth_model["cy"] - y_lo
        phi0 = _smooth_model["phi0"]
        dphi = 0.015
        phi_levels = [phi_peak - dphi, phi_peak, phi_peak + dphi]
        segs_phi = []
        for phi_lev in phi_levels:
            angle = phi_lev - phi0
            seg = _ray_segment_crop(cx_center, cy_center, angle, nx, ny)
            segs_phi.append(seg)
        dq = 0.5 * (float(np.nanmax(q_crop) - np.nanmin(q_crop)) or 1e-6) * 0.1
        dq = min(dq, 0.01)
        q_levels = [q_peak - dq, q_peak, q_peak + dq]
        segs_q = []
        for q_lev in q_levels:
            r = _r_from_q(_smooth_model, q_lev, r_max=r_max)
            if r is not None and r > 0:
                seg = _circle_segment_crop(cx_center, cy_center, r, nx, ny)
                segs_q.append(seg)
            else:
                segs_q.append(None)
        if len(segs_q) < 3:
            segs_q = [None] * 3
        q_levels = q_levels[:3]
    else:
        uq = np.unique(q_crop[np.isfinite(q_crop)])
        uphi = np.unique(phi_crop[np.isfinite(phi_crop)])
        q_range = float(np.nanmax(q_crop) - np.nanmin(q_crop)) or 1e-6
        if len(uq) >= 2:
            dq = 0.5 * float(np.min(np.diff(np.sort(uq))))
        else:
            dq = q_range * 0.05
        dq = min(dq, q_range * 0.5)
        q_levels = [q_peak - dq, q_peak, q_peak + dq]
        if len(uphi) >= 3:
            iphi = int(np.clip(np.argmin(np.abs(uphi - phi_peak)), 1, len(uphi) - 2))
            phi_levels = [uphi[iphi - 1], uphi[iphi], uphi[iphi + 1]]
        else:
            phi_levels = [phi_peak]
        segs_phi = [_contour_segment_through_peak(phi_crop, lev, cx, cy) for lev in phi_levels]
        segs_q = [_contour_segment_through_peak(q_crop, lev, cx, cy) for lev in q_levels]
    seg_phi = segs_phi[len(segs_phi) // 2] if segs_phi else None
    seg_q = segs_q[len(segs_q) // 2] if segs_q else None
    # Lineouts: use smooth (q, φ) from section 10 when available; I from image. Sample all 3 contours each.
    q_lineouts = []
    phi_lineouts = []
    for s in segs_phi:
        if s is not None:
            if use_smooth:
                q_phi, phi_phi, I_phi = _sample_along_segment_smooth(s, x_lo, y_lo, img_crop, _smooth_model)
            else:
                q_phi, phi_phi, I_phi = _sample_along_segment(s, q_crop, phi_crop, img_crop)
            if len(q_phi) > 0:
                idx = np.argsort(q_phi)
                q_lineouts.append((q_phi[idx], I_phi[idx]))
            else:
                q_lineouts.append((np.array([]), np.array([])))
        else:
            q_lineouts.append((np.array([]), np.array([])))
    for s in segs_q:
        if s is not None:
            if use_smooth:
                q_q, phi_q, I_q = _sample_along_segment_smooth(s, x_lo, y_lo, img_crop, _smooth_model)
            else:
                q_q, phi_q, I_q = _sample_along_segment(s, q_crop, phi_crop, img_crop)
            if len(phi_q) > 0:
                idx = np.argsort(phi_q)
                phi_lineouts.append((phi_q[idx], I_q[idx]))
            else:
                phi_lineouts.append((np.array([]), np.array([])))
        else:
            phi_lineouts.append((np.array([]), np.array([])))
    q_plot, I_q_plot = q_lineouts[len(q_lineouts) // 2][0], q_lineouts[len(q_lineouts) // 2][1]
    phi_plot, I_phi_plot = phi_lineouts[len(phi_lineouts) // 2][0], phi_lineouts[len(phi_lineouts) // 2][1]
    # Output lineouts and image for downstream cells (e.g. section 12 split Gaussian)
    LINEOUT_Q_X = np.asarray(q_plot, dtype=np.float64)
    LINEOUT_Q_I = np.asarray(I_q_plot, dtype=np.float64)
    LINEOUT_PHI_X = np.asarray(phi_plot, dtype=np.float64)
    LINEOUT_PHI_I = np.asarray(I_phi_plot, dtype=np.float64)
    LINEOUT_IMG_CROP = img_crop
    LINEOUT_VMIN = vmin
    LINEOUT_VMAX = vmax
    LINEOUT_SEG_PHI = seg_phi
    LINEOUT_SEG_Q = seg_q
    fig = plt.figure(figsize=(16, 8))
    gs = GridSpec(2, 2, height_ratios=[1.2, 1], hspace=0.35, wspace=0.3)
    ax_img = fig.add_subplot(gs[0, :])
    ax_q = fig.add_subplot(gs[1, 0])
    ax_phi = fig.add_subplot(gs[1, 1])
    im = ax_img.imshow(img_crop, origin="upper", cmap="magma", norm=LogNorm(vmin=vmin, vmax=vmax), interpolation="nearest", aspect="equal")
    ax_img.set_title("Temporal mean scattering (log scale)")
    ax_img.set_xticks([])
    ax_img.set_yticks([])
    for k, s in enumerate(segs_phi):
        if s is not None:
            lw = 1.5 if k == 1 else 1.0
            alpha = 0.95 if k == 1 else 0.7
            ax_img.plot(s[:, 0], s[:, 1], color="cyan", lw=lw, alpha=alpha)
    for k, s in enumerate(segs_q):
        if s is not None:
            lw = 1.5 if k == 1 else 1.0
            alpha = 0.95 if k == 1 else 0.7
            ax_img.plot(s[:, 0], s[:, 1], color="orange", lw=lw, alpha=alpha)
    fig.colorbar(im, ax=ax_img, label="Intensity")
    for k, (q_vals, I_vals) in enumerate(q_lineouts):
        if len(q_vals) > 0:
            lw = 1.5 if k == len(q_lineouts) // 2 else 1.0
            alpha = 0.95 if k == len(q_lineouts) // 2 else 0.7
            lab = f"φ ≈ {phi_levels[k]:.4f}" if k < len(phi_levels) else f"φ #{k}"
            ax_q.plot(q_vals, I_vals, "C0-", lw=lw, alpha=alpha, label=lab)
    ax_q.set_ylabel("Intensity")
    ax_q.set_yscale("log")
    ax_q.set_xlabel("q")
    ax_q.set_title("Q lineout (3 φ contours)")
    ax_q.legend(loc="upper right", fontsize=8)
    ax_q.grid(True, alpha=0.3)
    for k, (phi_vals, I_vals) in enumerate(phi_lineouts):
        if len(phi_vals) > 0:
            lw = 1.5 if k == len(phi_lineouts) // 2 else 1.0
            alpha = 0.95 if k == len(phi_lineouts) // 2 else 0.7
            lab = f"q ≈ {q_levels[k]:.4f}" if k < len(q_levels) else f"q #{k}"
            ax_phi.plot(phi_vals, I_vals, "C1-", lw=lw, alpha=alpha, label=lab)
    ax_phi.set_ylabel("Intensity")
    ax_phi.set_yscale("log")
    ax_phi.set_xlabel("φ")
    ax_phi.set_title("φ lineout (3 q contours)")
    ax_phi.legend(loc="upper right", fontsize=8)
    ax_phi.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

/var/folders/rf/wmzlz2kd779618zr2mgpkhhc0000gn/T/ipykernel_88968/1786830868.py:212: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## 12. Split Gaussian on q and φ lineouts

Fit split Gaussian to the lineout arrays from section 11. Plot data and fit together (linear scale), and print asymmetry from area on each side of the peak.

In [197]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from scipy.optimize import curve_fit

def _split_gaussian(x, A, mu, sigma_L, sigma_R):
    """Split Gaussian: left of mu uses sigma_L, right uses sigma_R. Continuous at peak."""
    x = np.asarray(x, dtype=np.float64)
    left = x <= mu
    out = np.empty_like(x)
    out[left] = A * np.exp(-0.5 * ((x[left] - mu) / np.clip(sigma_L, 1e-6, None)) ** 2)
    out[~left] = A * np.exp(-0.5 * ((x[~left] - mu) / np.clip(sigma_R, 1e-6, None)) ** 2)
    return out

def _fit_split_gaussian_1d_lineouts(x, y):
    """Fit split Gaussian; sigma scale matches x range (for q/φ in small units)."""
    y = np.asarray(y, dtype=np.float64)
    x = np.asarray(x, dtype=np.float64)
    x_range = float(np.nanmax(x) - np.nanmin(x)) or 1.0
    peak_idx = int(np.nanargmax(y))
    A0 = float(np.nanmax(y))
    mu0 = float(x[peak_idx])
    sig0 = x_range / 6.0
    sig_min = max(1e-8, x_range / 100.0)
    sig_max = max(x_range * 2.0, 1e-6)
    try:
        popt, _ = curve_fit(
            _split_gaussian, x, np.clip(y, 0, None),
            p0=[A0, mu0, sig0, sig0],
            bounds=([0, x.min(), sig_min, sig_min], [np.nanmax(y) * 2, x.max(), sig_max, sig_max]),
            maxfev=5000,
        )
        return popt
    except Exception:
        return None

try:
    _qx, _qi = LINEOUT_Q_X, LINEOUT_Q_I
    _px, _pi = LINEOUT_PHI_X, LINEOUT_PHI_I
except NameError:
    _qx = _qi = _px = _pi = None

if _qx is None or len(_qx) < 4:
    print("Run section 11 (Lineouts in q and φ) first to define LINEOUT_Q_X, LINEOUT_Q_I, LINEOUT_PHI_X, LINEOUT_PHI_I.")
else:
    fit_q = _fit_split_gaussian_1d_lineouts(_qx, _qi)
    fit_phi = _fit_split_gaussian_1d_lineouts(_px, _pi) if len(_px) >= 4 else None

    if fit_q is not None:
        A, mu, sigma_L, sigma_R = fit_q[0], fit_q[1], fit_q[2], fit_q[3]
        area_L = A * sigma_L * np.sqrt(np.pi / 2)
        area_R = A * sigma_R * np.sqrt(np.pi / 2)
        asym = (area_L - area_R) / (area_L + area_R) if (area_L + area_R) > 0 else 0.0
        print(f"Q lineout: area_L = {area_L:.4g}, area_R = {area_R:.4g}, asymmetry = {asym:.4f}")
    if fit_phi is not None:
        A, mu, sigma_L, sigma_R = fit_phi[0], fit_phi[1], fit_phi[2], fit_phi[3]
        area_L = A * sigma_L * np.sqrt(np.pi / 2)
        area_R = A * sigma_R * np.sqrt(np.pi / 2)
        asym = (area_L - area_R) / (area_L + area_R) if (area_L + area_R) > 0 else 0.0
        print(f"φ lineout: area_L = {area_L:.4g}, area_R = {area_R:.4g}, asymmetry = {asym:.4f}")

    fig = plt.figure(figsize=(16, 8))
    gs = GridSpec(2, 2, height_ratios=[1.2, 1], hspace=0.35, wspace=0.3)
    ax_img = fig.add_subplot(gs[0, :])
    ax_q = fig.add_subplot(gs[1, 0])
    ax_phi = fig.add_subplot(gs[1, 1])

    try:
        _img = LINEOUT_IMG_CROP
        _vmin, _vmax = LINEOUT_VMIN, LINEOUT_VMAX
        _seg_phi, _seg_q = LINEOUT_SEG_PHI, LINEOUT_SEG_Q
    except NameError:
        _img = _vmin = _vmax = _seg_phi = _seg_q = None
    if _img is not None:
        im = ax_img.imshow(_img, origin="upper", cmap="magma", norm=LogNorm(vmin=_vmin, vmax=_vmax), interpolation="nearest", aspect="equal")
        ax_img.set_title("Temporal mean scattering (log scale)")
        ax_img.set_xticks([])
        ax_img.set_yticks([])
        if _seg_phi is not None:
            ax_img.plot(_seg_phi[:, 0], _seg_phi[:, 1], color="cyan", lw=1.2, alpha=0.95)
        if _seg_q is not None:
            ax_img.plot(_seg_q[:, 0], _seg_q[:, 1], color="orange", lw=1.2, alpha=0.95)
        fig.colorbar(im, ax=ax_img, label="Intensity")

    ax_q.plot(_qx, _qi, "C0-", lw=1, label="I(q)")
    if fit_q is not None:
        ax_q.plot(_qx, _split_gaussian(_qx, *fit_q), "C1-", lw=1.2, label="split Gaussian")
        ax_q.annotate(f"σ_L = {fit_q[2]:.4g}  σ_R = {fit_q[3]:.4g}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=9, va="top")
    ax_q.set_ylabel("Intensity")
    ax_q.set_xlabel("q")
    ax_q.set_title("Q lineout")
    ax_q.legend(loc="upper right", fontsize=8)
    ax_q.grid(True, alpha=0.3)

    ax_phi.plot(_px, _pi, "C0-", lw=1, label="I(φ)")
    if fit_phi is not None:
        ax_phi.plot(_px, _split_gaussian(_px, *fit_phi), "C1-", lw=1.2, label="split Gaussian")
        ax_phi.annotate(f"σ_L = {fit_phi[2]:.4g}  σ_R = {fit_phi[3]:.4g}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=9, va="top")
    ax_phi.set_ylabel("Intensity")
    ax_phi.set_xlabel("φ")
    ax_phi.set_title("φ lineout")
    ax_phi.legend(loc="upper right", fontsize=8)
    ax_phi.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

Run section 11 (Lineouts in q and φ) first to define LINEOUT_Q_X, LINEOUT_Q_I, LINEOUT_PHI_X, LINEOUT_PHI_I.


## (Optional) Kernel memory usage

Run this cell anytime to see how much memory the current kernel process is using.

In [115]:
import os

try:
    import psutil
    p = psutil.Process(os.getpid())
    rss_mb = p.memory_info().rss / 1024**2
    print(f"Kernel memory (RSS): {rss_mb:.1f} MB")
except ImportError:
    import resource
    import sys
    ru = resource.getrusage(resource.RUSAGE_SELF)
    # maxrss: macOS = bytes, Linux = KB
    rss_mb = ru.ru_maxrss / 1024**2 if sys.platform == "darwin" else ru.ru_maxrss / 1024
    print(f"Kernel memory (max RSS): {rss_mb:.1f} MB")

Kernel memory (RSS): 97.6 MB
